# Interactive Risk Analysis Tool

## Purpose

This interactive tool allows risk managers to:
- Analyze specific markets in detail
- Adjust risk parameters and see immediate impact
- Run custom stress tests
- Export results for operational use

## How to Use

1. Select a market from the dropdown
2. Adjust the lookback period for historical analysis
3. Click "Fetch Data" to load current market data
4. Modify risk parameters to see how they affect position limits
5. Run stress tests to evaluate market resilience

In [ ]:
# Setup
import sys
sys.path.append('../src')
from risk_model import interactive_widgets as iw, notebook_helpers as nh
from risk_model import config, binance_data, pricing, scenarios
from risk_model.chart_config import setup_chart_style
from IPython.display import clear_output
import matplotlib.pyplot as plt

setup_chart_style()
markets_config = config.load_markets()
binance = binance_data.BinanceDataFetcher()

## Market Selection and Data Loading

Select the market you want to analyze and the historical lookback period. Longer periods provide more robust volatility estimates but may include outdated market conditions.

In [ ]:
# Create widgets
market_selector = iw.create_market_selector(markets_config['markets'])
lookback_slider = iw.create_lookback_slider(30)
fetch_button = iw.create_button('fetch')
output_area = iw.create_output_area()

# Store current data globally
current_data = {}

def fetch_data(b):
    global current_data
    with output_area:
        clear_output(wait=True)
        symbol = market_selector.value
        days = lookback_slider.value
        
        print(f"Fetching {days} days of data for {symbol}...")
        try:
            # Fetch all data
            data = nh.fetch_market_data([m for m in markets_config['markets'] 
                                       if m['binance_symbol_perp'] == symbol], 
                                      lookback_days=days)
            
            if data:
                current_data = list(data.values())[0]
                summary = nh.format_orderbook_summary(current_data['orderbook'])
                
                print("\nData loaded successfully!")
                print(f"Current Price: ${current_data['current_price']:,.2f}")
                print(f"24h Volume: ${current_data['volume_24h_usd']:,.0f}")
                print(f"Best Bid/Ask: ${summary['best_bid']:.2f} / ${summary['best_ask']:.2f}")
                print(f"Spread: {summary['spread_pct']:.3f}%")
                print(f"Volatility: {current_data['realized_vol']*100:.1f}%")
        except Exception as e:
            print(f"Error: {e}")

fetch_button.on_click(fetch_data)

# Display widgets
from ipywidgets import VBox, HBox
display(VBox([
    HBox([market_selector, lookback_slider]),
    fetch_button,
    output_area
]))

## Risk Parameter Configuration

### Parameter Definitions

- **Max Price Impact %**: Maximum acceptable slippage for liquidations
- **Max Leverage**: Highest leverage allowed for positions
- **OI/Liquidity Ratio**: Open interest as percentage of available liquidity
- **Position Limit %**: Single position as percentage of total OI

Adjust these parameters to see how they affect position limits and risk metrics.

In [ ]:
# Create risk parameter widgets
risk_params = iw.create_risk_parameter_widgets()
analysis_output = iw.create_output_area('500px')

def update_risk_analysis(impact, leverage, oi_ratio, position_limit):
    with analysis_output:
        clear_output(wait=True)
        
        if not current_data:
            print("Please fetch data first!")
            return
        
        # Recalculate limits
        depth = pricing.depth_at_impact(current_data['liquidity_curve'], impact)
        max_oi = depth * oi_ratio
        max_position = max_oi * position_limit
        
        # Display results table
        print("RISK ANALYSIS RESULTS")
        print("=" * 50)
        print(f"\nPosition Limits:")
        print(f"  Liquidity @ {impact}%: ${depth:,.0f}")
        print(f"  Max Total OI: ${max_oi:,.0f}")
        print(f"  Max Position Size: ${max_position:,.0f}")
        print(f"\nLeverage & Margin:")
        print(f"  Max Leverage: {leverage}x")
        print(f"  Initial Margin: {100/leverage:.1f}%")
        print(f"  Maintenance Margin: {100/leverage*0.5:.1f}%")
        print(f"\nMarket Quality:")
        print(f"  OI/Volume Ratio: {max_oi/current_data['volume_24h_usd']:.2f}x")
        print(f"  Liquidation Buffer: {100/leverage:.1f}%")
        
        # Create visualization
        fig, ax = plt.subplots(figsize=(10, 6))
        liq_curve = current_data['liquidity_curve']
        
        bid_data = liq_curve[liq_curve['side'] == 'bid']
        ask_data = liq_curve[liq_curve['side'] == 'ask']
        
        if not bid_data.empty:
            ax.plot(bid_data['price_impact_pct'], bid_data['cum_notional']/1e6,
                   'r-', label='Bids', linewidth=2)
        if not ask_data.empty:
            ax.plot(ask_data['price_impact_pct'], ask_data['cum_notional']/1e6,
                   'g-', label='Asks', linewidth=2)
        
        ax.axvline(impact, color='blue', linestyle='--', alpha=0.7, 
                  label=f'Impact Limit ({impact}%)')
        ax.axhline(max_oi/1e6, color='orange', linestyle='--', alpha=0.7,
                  label=f'Max OI (${max_oi/1e6:.1f}M)')
        
        ax.set_xlabel('Price Impact (%)')
        ax.set_ylabel('Liquidity (USD millions)')
        ax.set_title(f"{current_data['symbol']} - Liquidity Analysis")
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Create interactive widget
from ipywidgets import interactive
interactive_analysis = interactive(
    update_risk_analysis,
    impact=risk_params['impact'],
    leverage=risk_params['leverage'],
    oi_ratio=risk_params['oi_ratio'],
    position_limit=risk_params['position_limit']
)

display(VBox([
    iw.HTML('<h3>Adjust Risk Parameters</h3>'),
    interactive_analysis,
    analysis_output
]))

## Stress Testing

### Scenario Descriptions

- **Liquidation Scenarios**: Simulate mass liquidations of X% of open interest
- **Price Gap Scenarios**: Evaluate impact of sudden price moves
- **Cascade Effects**: Model sequential liquidations

Select a scenario and run the stress test to see potential market impact.

In [ ]:
# Stress testing widgets
scenario_selector = iw.create_stress_test_selector()
stress_button = iw.create_button('stress_test')
stress_output = iw.create_output_area('400px')

def run_stress_test(b):
    with stress_output:
        clear_output(wait=True)
        
        if not current_data:
            print("Please fetch data first!")
            return
        
        scenario_type = scenario_selector.value
        
        # Get current parameters
        depth = pricing.depth_at_impact(current_data['liquidity_curve'], 
                                       risk_params['impact'].value)
        max_oi = depth * risk_params['oi_ratio'].value
        
        print(f"STRESS TEST: {scenario_selector.label}")
        print("=" * 50)
        
        if 'liquidation' in scenario_type:
            # Extract percentage
            pct = int(scenario_type.split('_')[1]) / 100
            result = scenarios.liquidation_impact_scenario(
                max_oi, current_data['liquidity_curve'], pct
            )
            
            print(f"\nLiquidation Impact:")
            print(f"  Size: ${result['liquidation_size_usd']:,.0f}")
            print(f"  Bid Impact: {result['bid_impact_pct']:.2f}%")
            print(f"  Ask Impact: {result['ask_impact_pct']:.2f}%")
            print(f"  Average Impact: {result['avg_impact_pct']:.2f}%")
            print(f"  Severity: {result['severity'].upper()}")
            
        elif 'gap' in scenario_type:
            # Extract gap percentage
            gap = int(scenario_type.split('_')[1])
            result = scenarios.gap_risk_scenario(
                current_data['current_price'], gap,
                1/risk_params['leverage'].value * 0.5,
                max_oi, current_data['total_liquidity_usd']
            )
            
            print(f"\nPrice Gap Analysis:")
            print(f"  Gap Size: {result['gap_pct']}%")
            print(f"  New Price: ${result['gap_price']:,.2f}")
            print(f"  Positions Underwater: {result['positions_underwater_pct']:.1f}%")
            print(f"  Potential Liquidations: ${result['potential_liquidations_usd']:,.0f}")
            print(f"  Market Can Absorb: {'YES' if result['can_absorb'] else 'NO'}")
            print(f"  Severity: {result['severity'].upper()}")
        
        # Recommendations
        print("\nRecommendations:")
        if result['severity'] == 'high':
            print("  - Consider reducing position limits")
            print("  - Implement circuit breakers")
            print("  - Increase margin requirements")
        elif result['severity'] == 'medium':
            print("  - Monitor market conditions closely")
            print("  - Review risk parameters weekly")
        else:
            print("  - Current parameters appear adequate")
            print("  - Continue regular monitoring")

stress_button.on_click(run_stress_test)

display(VBox([
    iw.HTML('<h3>Stress Testing</h3>'),
    HBox([scenario_selector, stress_button]),
    stress_output
]))

## Export Results

Export your analysis results for operational use or reporting. The export includes:
- Current market data
- Risk parameters used
- Calculated limits
- Stress test results

In [ ]:
# Export functionality
import json
from datetime import datetime

def export_analysis():
    if not current_data:
        print("No data to export. Please fetch data first.")
        return
    
    # Gather all parameters and results
    export_data = {
        'timestamp': datetime.now().isoformat(),
        'market': current_data['symbol'],
        'current_price': current_data['current_price'],
        'volume_24h': current_data['volume_24h_usd'],
        'volatility': current_data['realized_vol'],
        'parameters': {
            'max_impact_pct': risk_params['impact'].value,
            'max_leverage': risk_params['leverage'].value,
            'oi_liquidity_ratio': risk_params['oi_ratio'].value,
            'position_limit_pct': risk_params['position_limit'].value
        },
        'limits': {
            'max_oi_usd': pricing.depth_at_impact(current_data['liquidity_curve'], 
                                                 risk_params['impact'].value) * risk_params['oi_ratio'].value,
            'max_position_usd': pricing.depth_at_impact(current_data['liquidity_curve'], 
                                                       risk_params['impact'].value) * 
                               risk_params['oi_ratio'].value * risk_params['position_limit'].value
        }
    }
    
    # Save to file
    filename = f"risk_analysis_{current_data['symbol']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(f'../../reports/{filename}', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"Analysis exported to: reports/{filename}")
    print("\nExported data summary:")
    print(json.dumps(export_data, indent=2))

# Export button
export_button = iw.create_button('export')
export_button.on_click(lambda b: export_analysis())

display(VBox([
    iw.HTML('<h3>Export Analysis</h3>'),
    export_button
]))

## Additional Analysis

Use this section for any custom analysis or visualization you need. The `current_data` dictionary contains all fetched market data.

In [ ]:
# Custom analysis space
# Access current_data dictionary for any additional analysis
# Example: current_data['klines'], current_data['orderbook'], etc.